In [1]:
# Librerias
import pandas as pd
import re

In [2]:
## Cargar el archivo

# Definir la ruta del archivo
archivo = "Actividad - 10,000.xlsx"  # Cambia esta ruta si es necesario
df = pd.read_excel(archivo, engine='openpyxl')
# Verificar las primeras filas del archivo
df.head(10)


,Apartado,Tipo,Bien,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,Mobiliario,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO COLOR NEGRO. N...,NaN,NaN,NaN,NaN,NaN
1,Mobiliario,Mobiliario,PINTARRON 60X90 (ALTA 20 ENE 1999 MARCA ALFRA).,NaN,NaN,NaN,NaN,NaN
2,Equipo,Equipo,"MONITOR DE COLOR HP DE 15"", N.D. 017/2002 ALTA...",NaN,NaN,NaN,NaN,NaN
3,Mobiliario,Mobiliario,"CESTO DE BASURA ALUMINIO N.D. 007/2005, ALTA 3...",NaN,NaN,NaN,NaN,NaN
4,Transporte,Transporte,"AUTOMOVIL FORD ESCORT 1998, PLACAS: HZG1129, N...",NaN,NaN,NaN,NaN,NaN
5,Equipo,Equipo,"TRITURADORA DE DOCUMENTOS. ALTA 13 SEP 2006, N...",NaN,NaN,NaN,NaN,NaN
6,Mobiliario,Mobiliario,SILLON P/SALA RECEPCION.(SILLON FIJO III PLAZA...,NaN,NaN,NaN,NaN,NaN
7,Equipo,Equipo,REGULADOR TDE S/N: 16030085976,NaN,NaN,NaN,NaN,NaN
8,Equipo,Equipo,IDENTIFICADOR DE LLAMADAS,NaN,NaN,NaN,NaN,NaN
9,Mobiliario,Mobiliario,MESA PARA IMPRESORA CON MEDIDA DE .80 DE FRENT...,NaN,NaN,NaN,NaN,NaN


In [3]:
## Elimiannar redundancias y renombrar columnas

columnas_a_eliminar = ['Unnamed: 3','Unnamed: 4', 'Unnamed: 5', 'Unnamed: 6', 'Unnamed: 7']
df = df.drop(columns=[col for col in columnas_a_eliminar if col in df.columns])

df.head(1)
# Eliminar la columna "Tipo"
if 'Tipo' in df.columns:
    df.drop('Tipo', axis=1, inplace=True)

# Renombrar la columna "Bien" a "Descripcion"
df.rename(columns={'Bien': 'Descripcion'}, inplace=True)

df.head(1)


,Apartado,Descripcion
0,Mobiliario,SILLON EJECUTIVO RESPALDO MEDIO COLOR NEGRO. N...


In [4]:
# Limpieza de la columna 'Descripcion' 

# Tupla de palabras irrelevantes para eliminar del texto
palabras_irrelevantes = (
    'color', 'numero', 'serie', 'tamaño', 'alto', 'ancho', 'profundidad',
    'marca', 'modelo', 'placa', 'nd', 'de', 'la', 'el', 'los', 'las', 'es', 'base', 'equipo',
    'un', 'una', 'por', 'para', 'con', 'en', 'del', 'tipo', 'y', 'o',
    'características', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9'
)
palabras_irrelevantes_set = set(palabras_irrelevantes)  # Convertir la tupla a conjunto para búsquedas rápidas

# Función para limpiar y corregir texto
def limpiar_texto(texto):
    if isinstance(texto, str):
        # Convertir el texto a minúsculas
        texto = texto.lower()
        # Eliminar caracteres especiales, pero conservar números y espacios
        texto = re.sub(r'[^a-záéíóúñü0-9\s]', '', texto)
        # Eliminar palabras irrelevantes junto con el espacio a la derecha si existe
        for palabra_irrelevante in palabras_irrelevantes_set:
            texto = re.sub(rf'\b{palabra_irrelevante}\b\s*', '', texto)
        return texto
    else:
        return ''  # Si no es texto, devolver cadena vacía


# Aplicar la función de limpieza y corrección a la columna "Descripcion"
df['Descripcion'] = df['Descripcion'].apply(limpiar_texto)


# Mostrar los primeros registros procesados
df[['Descripcion']].head()


,Descripcion
0,sillon ejecutivo respaldo medio negro 132002 a...
1,pintarron 60x90 alta 20 ene 1999 alfra
2,monitor hp 15 0172002 alta 07 octubre 2002
3,cesto basura aluminio 0072005 alta 31 agosto 2005
4,automovil ford escort 1998 placas hzg1129 no 3...


In [5]:
def normalizar_descripcion(descripcion, terminos_reemplazo):
    """
    Normaliza los términos en la columna de descripción según los términos y sus reemplazos dados.

    Args:
        descripcion (str): La descripción a normalizar.
        terminos_reemplazo (dict): Diccionario con términos como clave y sus reemplazos como valor. 
                                    Tanto la clave como el valor deben estar en minúsculas.

    Returns:
        str: La descripción normalizada.
    """
    # Convertir la descripción a minúsculas para hacerla uniforme
    descripcion = descripcion.lower()
    
    # Reemplazar cada término en la descripción con su equivalente normalizado
    for termino, reemplazo in terminos_reemplazo.items():
        descripcion = re.sub(rf'\b{re.escape(termino)}\b', reemplazo, descripcion)
    
    return descripcion

# Definir el diccionario de términos a normalizar
terminos_reemplazo = {
    'c.p.u': 'cpu',
    'cpu': 'cpu',
    # Agregar más términos según sea necesario
}

# Aplicar la función de normalización a la columna 'Descripcion'
df['Descripcion'] = df['Descripcion'].apply(normalizar_descripcion, terminos_reemplazo=terminos_reemplazo)


In [6]:
# Nueva columna "Item"

# Definir palabras clave para la extracción de "Item" y compilarlas
palabras_clave = ['silla', 'mesa', 'monitor', 'computadora', 'archivero', 'control', 'automovil', 'sofá']
pattern_palabras_clave = re.compile(r'\b(' + '|'.join(palabras_clave) + r')\b')

# Función para extraer el "Item" basado en palabras clave
def extraer_item(descripcion):
    if isinstance(descripcion, str) and descripcion:
        match = pattern_palabras_clave.search(descripcion)
        if match:
            return match.group(1)
        else:
            # Fallback: primera palabra
            return descripcion.split()[0]
    return ''

# Aplicar la extracción de "Item"
df['Item'] = df['Descripcion'].apply(extraer_item)

df.head(1)

,Apartado,Descripcion,Item
0,Mobiliario,sillon ejecutivo respaldo medio negro 132002 a...,sillon


In [7]:
# Asegurarnos de que todos los valores de la columna 'Descripcion' sean cadenas de texto y rellenar valores nulos
df['Descripcion'] = df['Descripcion'].astype(str).fillna('')

In [8]:
# Crear una lista de categorías para clasificar 'Tipo_Articulo' más detalladamente
def clasificar_tipo_articulo(descripcion, item):
    descripcion = descripcion.lower()
    item = item.lower()
    
    # Definiciones de categorías
    if any(x in descripcion for x in ['mesa', 'silla', 'pintarron', 'archivador', 'escritorio']):
        return 'Oficina'
    elif any(x in descripcion for x in ['comedor', 'cocina', 'vajilla']):
        return 'Cocina'
    elif any(x in descripcion for x in ['cesto', 'papelera']):
        return 'Limpieza'
    elif any(x in descripcion for x in ['impresora', 'monitor', 'laptop', 'computadora', 'teclado']):
        return 'Oficina'
    elif any(x in descripcion for x in ['automovil', 'motocicleta', 'bicicleta', 'camioneta']):
        return 'Transporte'
    elif any(x in descripcion for x in ['refrigerador', 'boiler', 'microondas', 'calentador']):
        return 'Electrodoméstico'
    elif any(x in descripcion for x in ['herramienta', 'taladro', 'trituradora', 'martillo']):
        return 'Herramienta'
    elif 'mueble' in item:
        return 'Mueble - General'
    elif 'equipo' in item:
        return 'Equipo'
    else:
        return 'No Aplica'

# Aplicar la clasificación mejorada a la columna 'Tipo_Articulo'
df['Tipo_Articulo'] = df.apply(lambda row: clasificar_tipo_articulo(row['Descripcion'], row['Item']), axis=1)

In [9]:
# En base a la fecha y descripcion de antiguedad
def clasificar_antiguedad(descripcion):
    # Extraer año de la descripción
    match = re.search(r'\b(19|20)\d{2}\b', descripcion)
    if match:
        year = int(match.group())
        current_year = pd.Timestamp.now().year
        antiguedad = current_year - year

        # Clasificar según el rango de años
        if antiguedad <= 2:
            return 'Alta Reciente'
        elif 3 <= antiguedad <= 10:
            return 'Alta Intermedia'
        else:
            return 'Alta Antigua'
    else:
        return 'Desconocido'

# Aplicar la clasificación mejorada de antigüedad
df['Antiguedad'] = df['Descripcion'].apply(clasificar_antiguedad)

In [10]:
def clasificar_uso(descripcion, item):
    # Convertir a minúsculas para una evaluación insensible a mayúsculas
    descripcion = descripcion.lower()
    item = item.lower()

    # Definición de categorías y palabras clave
    categorias = {
        'Oficina': ['oficina', 'escritorio', 'silla', 'monitor', 'papelera', 'lámpara', 'archivador', 'impresora'],
        'Cocina': ['comedor', 'cocina', 'electrodoméstico', 'nevera', 'microondas', 'refrigerador', 'cafetera'],
        'Limpieza': ['cesto', 'limpieza', 'escoba', 'recogedor', 'detergente', 'aspiradora', 'trapeador'],
        'Transporte': ['transporte', 'vehículo', 'bicicleta', 'coche', 'moto'],
        'Almacenamiento': ['almacenamiento', 'archivo', 'estante', 'cajón', 'contenedor'],
        'Tecnología': ['comunicación', 'electrónico', 'tecnología', 'móvil', 'tablet', 'computadora', 'ratón'],
        'Mobiliario': ['mobiliario', 'mueble', 'soporte', 'gabinete'],
        'Decoración': ['decoración', 'cuadro', 'florero', 'lámpara', 'alfombra', 'espejo'],
        'Construcción': ['construcción', 'herramienta', 'martillo', 'taladro', 'destornillador'],
        'Jardinería': ['jardín', 'jardinería', 'tijeras', 'pala', 'manguera'],
        'Deporte': ['deporte', 'equipamiento', 'balón', 'raqueta', 'pesa', 'bici'],
        'Electrodoméstico': ['electrodoméstico', 'lavadora', 'secadora', 'horno'],
        'Entretenimiento': ['juego', 'videojuego', 'consola', 'televisor', 'película'],
    }

    # Evaluar las categorías en base a descripción e item
    for categoria, palabras_clave in categorias.items():
        if any(palabra in descripcion or palabra in item for palabra in palabras_clave):
            return categoria

    # Si no se encuentra ninguna coincidencia, retornar 'No Aplica'
    return 'No Aplica'

# Aplicar la clasificación mejorada de uso
df['Uso'] = df.apply(lambda row: clasificar_uso(row['Descripcion'], row['Item']), axis=1)


In [11]:

# Guardar el archivo procesado con la columna limpia
df.to_excel("Actividad_Limpia.xlsx", index=False)

# Mostrar una confirmación
print("Archivo guardado como Actividad_Limpia.xlsx")


Archivo guardado como Actividad_Limpia.xlsx
